
# Illustrates nested Loop Joins and lateral joins in SQL
  
cs3200: Database design  
https://northeastern-datalab.github.io/cs3200/  

CS 7240: Principles of scalable data management  
https://northeastern-datalab.github.io/cs7240/  

CS 7575: A Seminar On Relational Language Design    
https://northeastern-datalab.github.io/cs7575/  

SQL files 305, 720, 735  
https://github.com/northeastern-datalab/cs3200-activities/tree/master/sql    

First version: 1/26/2026  
This version: 2/18/2026  

# A. Understanding conceptual evaluation strategy with disjunction and empty table
SQL file 305

In [59]:
X = [1,2]
Y = [1,2,3]
Z = [1]
# Z = []   # comment out to see the difference


print("--- nested loop with disjunction ---")
for x in X:
    for y in Y:
        for z in Z:
            cond = "True" if (x == y or x == z) else ""
            print(f"x={x}, y={y}, z={z}: {cond}")            

--- nested loop with disjunction ---
x=1, y=1, z=1: True
x=1, y=2, z=1: True
x=1, y=3, z=1: True
x=2, y=1, z=1: 
x=2, y=2, z=1: True
x=2, y=3, z=1: 


# B. Inner joins vs. lateral joins vs. nested correlated queries
SQL file 735

In [60]:
# Data

X = [1, 2, 3, 4]        # X = range(1, 5)
Y = [1, 2, 3, 4]

## 1. Inner joins
Standard nested loop strategy for inner joins: the sequence of the loops does not matter

In [61]:
result = [(x, y) for x in X for y in Y if x < y]    # list comprehension: builds everything immediately

print(type(result))

for row in result:
    print(row)

<class 'list'>
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [62]:
result = ((x, y) for x in X for y in Y if x < y)    # generator expression: yields items lazily, one at a time

print(type(result))

for row in result:
    print(row)

<class 'generator'>
(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [63]:
result = []
for x in X:
    for y in Y:
        if x < y:
            result.append((x, y))   # creates a list
            
for row in result:
    print(row)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [64]:
def generate_pairs(X, Y):           # as generator function that generates output pairs on demand
    for x in X:
        for y in Y:
            if x < y:
                yield (x, y)        # any function defined with "def" and that contains "yield" defines a generator

for row in generate_pairs(X, Y):
    print(row)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [65]:
for x in X:
  for y in Y:
    cond = "True" if (x<y) else ""
    print(f"({x}, {y}): {cond}")

(1, 1): 
(1, 2): True
(1, 3): True
(1, 4): True
(2, 1): 
(2, 2): 
(2, 3): True
(2, 4): True
(3, 1): 
(3, 2): 
(3, 3): 
(3, 4): True
(4, 1): 
(4, 2): 
(4, 3): 
(4, 4): 


## 2. Lateral join

In [66]:
result = [(x, z) for x in X for z in (y for y in Y if x < y)]

for row in result:
    print(row)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


In [67]:
for x in X:
  Z = [y for y in Y if x < y]
  for z in Z:
    cond = "True" if (x<y) else ""
    print(f"({x}, {z}): {cond}")

(1, 2): True
(1, 3): True
(1, 4): True
(2, 3): True
(2, 4): True
(3, 4): True


## 3. Nested correlated query

In [68]:
result = [x for x in X if [y for y in Y if x < y]]      # condition true if the inner list is not empty

for row in result:
    print(row)

1
2
3


In [69]:
result = [x for x in X if any(x < y for y in Y)]        # better: stops at first match

for row in result:
    print(row)

1
2
3


In [70]:
for x in X:
  cond = "True" if any(x < y for y in Y) else ""
  print(f"{x}: {cond}")

1: True
2: True
3: True
4: 


## 4.1 Grouped: SUM/MIN

In [71]:
X = [1, 2, 3, 4]        # X = range(1, 5)
Y = [1, 2, 3, 4]

In [72]:
# SUM

from collections import defaultdict
agg = defaultdict(lambda: 0)    # maps a -> sum of matching b's; If a key does not exist, call the function to return 0 to create a default value 0

for a in X:
    for b in Y:
        if a < b:
            agg[a] += b     # first time: agg[a] is auto-created as 0, then += b

result = [(a, agg[a]) for a in agg]     # "for a in agg" same as "for a in agg.keys()" 
# result = list(agg.items())            # cleaner: list(agg.items()) directly gives (key, value) pairs.

for row in result:
    print(row)

(1, 9)
(2, 7)
(3, 4)


In [73]:
# SUM / MIN

from collections import defaultdict
import math

aggs = defaultdict(lambda: 0)           # maps a -> sum of matching b's; If a key does not exist, call the function to return 0 to create a default value 0
aggm = defaultdict(lambda: math.inf)    # maps a -> sum of matching b's; If a key does not exist, call the function to return 0 to create a default value 0

for a in X:
    for b in Y:
        if a < b:
            aggs[a] += b                # first time: aggs[a] = 0; agg[a] = agg[a] + b
            aggm[a] = min(aggm[a], b)   # first time: aggm[a] = math.inf; agg[a] = agg[a] + b

result = [(a, aggs[a], aggm[a]) for a in aggs]   # "for a in aggs" same as "for a in aggs.keys()" 

for row in result:
    print(row)

(1, 9, 2)
(2, 7, 3)
(3, 4, 4)


In [74]:
# PANDAS SUM / MIN

import pandas as pd

dfX = pd.DataFrame({"A": list(X)})  # in case X is a generator and gets consumed during the pandas build
dfY = pd.DataFrame({"B": list(Y)})

# cross join + filter + groupby
result = (
    dfX.merge(dfY, how="cross")
       .query("A < B")
       .groupby("A", as_index=False)
       .agg(
            C=("B", "sum"),
            D=("B", "min")
       )
)
print(result)

   A  C  D
0  1  9  2
1  2  7  3
2  3  4  4


In [75]:
# SUM like outer joins with 0

from collections import defaultdict
agg = defaultdict(lambda: 0)    # maps a -> sum of matching b's; If a key does not exist, call the function to return 0 to create a default value 0

for a in X:
    agg[a] = 0  # ensure all X values appear (LEFT JOIN behavior)
    
    for b in Y:
        if a < b:
            agg[a] += b     # first time: agg[a] is auto-created as 0, then += b

result = [(a, agg[a]) for a in agg]     # "for a in agg" same as "for a in agg.keys()" 
# result = list(agg.items())            # cleaner: list(agg.items()) directly gives (key, value) pairs.

for row in result:
    print(row)

(1, 9)
(2, 7)
(3, 4)
(4, 0)


In [76]:
# SUM like outer joins with null

from collections import defaultdict
agg = defaultdict(lambda: 0)    # maps a -> sum of matching b's; If a key does not exist, call the function to return 0 to create a default value 0

for a in X:
    agg[a] = None  # LEFT JOIN default (NULL)
    
    for b in Y:
        if a < b:
            agg[a] = b if agg[a] is None else agg[a] + b

result = [(a, agg[a]) for a in agg]     # "for a in agg" same as "for a in agg.keys()" 
# result = list(agg.items())            # cleaner: list(agg.items()) directly gives (key, value) pairs.

for row in result:
    print(row)

(1, 9)
(2, 7)
(3, 4)
(4, None)


## 4.2 Array Aggregate

In [77]:
X = [1, 2, 3, 4]        # X = range(1, 5)
Y = [1, 2, 3, 4]

In [78]:
# Array aggregate

from collections import defaultdict
agg = defaultdict(list) 

for a in X:
    for b in Y:
        if a < b:
            agg[a].append(b)   # collect values

result = [(a, agg[a]) for a in agg]     # "for a in agg" same as "for a in agg.keys()" 
# result = list(agg.items())            # cleaner: list(agg.items()) directly gives (key, value) pairs.

for row in result:
    print(row)

(1, [2, 3, 4])
(2, [3, 4])
(3, [4])


In [79]:
# Unnest again

Z = result

output = []
for a, c in Z:
  W = [w for w in c]
  for w in W:
    output.append((a, w))

for row in output:
    print(row)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)


# C Lateral join example
SQL file 720 